    On REAL-TIME project, dont rely on the template rather apply the concept from this.

In [ ]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, train_test_split

# Visualisation
from sklearn import tree
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Declare feature vector and target variable
X = df.drop(['target'], axis = 1)

y = df['target']

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [ ]:
dfc_model1 = DecisionTreeClassifier().fit(xtrain, ytrain)

In [ ]:
# Create a DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier()

# Define the hyperparameter grid for GridSearchCV
params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['auto', 'sqrt', 'log2'],
    'random_state': [42]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=dt_classifier, param_grid=params, cv=4, n_jobs=-1, verbose=1, scoring="accuracy")

# Fit the grid search to your training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Check the best score
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score (Grid Search):", best_score)

In [ ]:
final_dtc_model.fit(X_train, y_train)

dtc_predicted_y_train = final_dtc_model.predict(X_train)
dtc_Train_accuracy = accuracy_score(y_train, dtc_predicted_y_train)

dtc_predicted_y_test = final_dtc_model.predict(X_test)
dtc_Train_accuracy = accuracy_score(y_test, dtc_predicted_y_test)

print(f"Train Accuraccy : {dtc_Train_accuracy}")
print(f"Train Accuraccy : {dtc_Train_accuracy}")

In [ ]:
alpha_dict  = final_dtc_model.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = alpha_dict["ccp_alphas"]
impurities = alpha_dict["impurities"]
print(f'ccp_alpha : {ccp_alphas}\n Impurities : {impurities}')

In [ ]:
test_accuracy_1 = []
train_accuracy_1 = []
for alpha in ccp_alphas:
    
    Model = DecisionTreeClassifier(ccp_alpha = alpha, criterion = 'entropy', max_depth = 10, max_features = 'auto', min_samples_leaf = 2, min_samples_split = 2, random_state = 42).fit(X_train, y_train)
    pred_test = Model.predict(X_test)
    test_accuracy = accuracy_score(y_test, pred_test)
    
    pred_train = Model.predict(X_train)
    train_accuracy = accuracy_score(y_train, pred_train)
    
    train_accuracy_1.append(train_accuracy)
    test_accuracy_1.append(test_accuracy)
    
df_ccp_alpha = pd.DataFrame()
df_ccp_alpha['train_accuracy_1'] = train_accuracy_1
df_ccp_alpha['test_accuracy_1'] = test_accuracy_1
df_ccp_alpha['Alpha'] = ccp_alphas
df_ccp_alpha['gap'] = abs(df_ccp_alpha['train_accuracy_1'] - df_ccp_alpha['test_accuracy_1'])
df_ccp_alpha.head()

In [ ]:
plt.plot(df_ccp_alpha['test_accuracy_1'], color = 'red', marker = '*')
plt.plot(df_ccp_alpha['train_accuracy_1'], color = 'green', marker = '*')

In [ ]:
filtered_df = df_ccp_alpha[(df_ccp_alpha['train_accuracy_1'] > 0.9) & (df_ccp_alpha['test_accuracy_1'] > 0.9)]

# If there are no rows matching the condition, you can handle that case
if filtered_df.empty:
    print("No rows meet the criteria.")
else:
    # Find the row with the least gap
    min_gap_row = filtered_df[filtered_df['gap'] == filtered_df['gap'].min()]
    
    # Get the corresponding alpha value
    corresponding_alpha = min_gap_row['Alpha'].iloc[0]
    
    print(f"The least gap is {min_gap_row['gap'].iloc[0]} with Accuracy of which corresponds to  Alpha value {corresponding_alpha}")
min_gap_row

In [ ]:
model_tree_2 = DecisionTreeClassifier(ccp_alpha = 0.003457216940363008, criterion = 'entropy', max_depth = 10, max_features = 'auto', min_samples_leaf = 2, min_samples_split = 2, random_state = 42).fit(X_train, y_train)

# Check whether how this model fits the data
predicted_y_train2 = model_tree_2.predict(X_train)
train_accuracy_2 = accuracy_score(y_train, predicted_y_train2)

predicted_y_test2 = model_tree_2.predict(X_test)
test_accuracy_2 = accuracy_score(y_test, predicted_y_test2)

print(f"Train Accuraccy : {train_accuracy_2}")
print(f"Train Accuraccy : {test_accuracy_2}")

In [ ]:
plt.figure(figsize = (20,20))
tree.plot_tree(model_tree_2, feature_names = X_train.columns)
plt.show()